In [ ]:
using Revise
using CUDA
using DataFrames
using JSON
using Statistics
using Juliana

# Config

In [ ]:
# gantry_angle = 36f0
# couch_angle = 64f0
gantry_angle = 0f0
couch_angle = 0f0
nozzle_extraction = 15f0
preabsorber = "OUT"

In [ ]:
fiona_standalone_bin_path = "/data/user/bellotti_r/semester_project_planning_metrics/src/pyftpp/bin"
fiona_jar_path = "$fiona_standalone_bin_path/ch.psi.ftpp.standalone.planner-1.0.10.jar"; # Test works with 1.0.7, but not 1.0.10...

In [ ]:
output_dir = "../output/test/dose_calc/full_plan"

mkpath(output_dir)

# Load data

In [ ]:
# ct, target = Juliana.build_water_slab()
# ct_path = "$(output_dir)/ct.dat"
# Juliana.write_ct_dat_file(ct_path, ct)

# patient_ID = "bellotti_r_water_phantom";

In [ ]:
patient_ID = "test_00"
data_dir = "/data/user/bellotti_r/data"
ct_path, patient = Juliana.load_patient_data(data_dir, patient_ID)

ct = patient.ct
target = patient.structures["PTV2=CTV2+5MM"];

In [ ]:
optimisation_points = Juliana.mask_to_points(
    ct.grid,
    target.mask,
)

optimisation_grid = Juliana.get_optimisation_grid(
    optimisation_points',
    ct.grid,
)

# Optimise the spots and calculate the dose using Fiona

In [ ]:
tps = Juliana.FionaStandalone.FionaStandaloneTps(
    fiona_standalone_bin_path,
    fiona_jar_path,
    output_dir,
);

In [ ]:
plan = Juliana.FionaStandalone.place_spots(
    tps,
    ct,
    target,
    preabsorber,
    [gantry_angle],
    [couch_angle],
    [nozzle_extraction],
)
w = DataFrame(only(plan.fields).spots).weight;

In [ ]:
tps = Juliana.FionaStandalone.FionaStandaloneTps(
    fiona_standalone_bin_path,
    fiona_jar_path,
    output_dir,
)
Dij_fiona = Juliana.FionaStandalone.calculate_Dij_matrix(tps, ct, plan, optimisation_points, ct_path);
dose_flat_fiona = reshape(
    Dij_fiona * w,
    (size(Dij_fiona, 1), 1),
);

In [ ]:
dose_fiona = zeros(Float32, Tuple(ct.grid.size));

for (i, p) in enumerate(eachrow(optimisation_points'))
    indices = Juliana.xyz_to_index(p, ct.grid)
    dose_fiona[indices...] = dose_flat_fiona[i]
end

# Calculate dose using Juliana

In [ ]:
tps = Juliana.load_juliana_tps(fiona_standalone_bin_path, nozzle_extraction);

In [ ]:
Dij_juliana = Juliana.calculate_Dij_matrix(tps, ct, plan, optimisation_points);

In [ ]:
dose_juliana = zeros(Float32, Tuple(ct.grid.size));

tmp = collect(Dij_juliana) * w

for (i, p) in enumerate(eachrow(collect(optimisation_points)'))
    indices = Juliana.xyz_to_index(p, ct.grid)
    dose_juliana[indices...] = tmp[i]
end

# Plot

In [ ]:
maximum(abs.(dose_fiona .- dose_juliana))

In [ ]:
maximum(abs.(dose_fiona .- dose_juliana)) / maximum(dose_fiona) * 100

In [ ]:
sum(dose_fiona)

In [ ]:
sum(dose_juliana)

In [ ]:
maximum(dose_juliana)

In [ ]:
normalisation_dose = max(maximum(dose_juliana), maximum(dose_fiona))

In [ ]:
Juliana.plot_distributions(
    ct,
    [("Fiona", dose_fiona), ("Juliana", dose_juliana)],
    [],
    # 1, 100,
    # 1, 100,
    # 12,
    100, 160,
    100, 160,
    39,
    normalisation_dose,
    Juliana.build_colorscheme(),
)

# Juliana.plot_distributions(
#     ct,
#     [("Fiona", dose_fiona), ("Juliana", dose_juliana)],
#     [],
#     1, 100,
#     1, 100,
#     12,
#     normalisation_dose,
#     Juliana.build_colorscheme(),
# )

In [ ]:
maximum(abs.(dose_fiona .- dose_juliana)) ./ maximum(dose_fiona) * 100

In [ ]:
maximum(dose_fiona)